In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

np.random.seed(17)
tf.random.set_seed(17)

In [2]:
densenet = tf.keras.applications.DenseNet201()

In [3]:
densenet.summary(line_length=200)

Model: "densenet201"
________________________________________________________________________________________________________________________________________________________________________________________________________
 Layer (type)                                                     Output Shape                                Param #                 Connected to                                                      
 input_1 (InputLayer)                                             [(None, 224, 224, 3)]                       0                       []                                                                
                                                                                                                                                                                                        
 zero_padding2d (ZeroPadding2D)                                   (None, 230, 230, 3)                         0                       ['input_1[0][0]']                        

In [4]:
cat_img = tf.keras.preprocessing.image.load_img('cat.jpg', target_size=(224, 224))

In [5]:
#cat_img.show()

In [6]:
cat_img = tf.keras.preprocessing.image.img_to_array(cat_img)

In [7]:
cat_img.shape

(224, 224, 3)

In [8]:
cat_img = np.expand_dims(cat_img, axis=0)

In [9]:
cat_img.shape

(1, 224, 224, 3)

In [10]:
cat_img = tf.keras.applications.densenet.preprocess_input(cat_img)

In [11]:
perd = densenet.predict(cat_img)

In [12]:
tf.keras.applications.densenet.decode_predictions(perd)

[[('n02124075', 'Egyptian_cat', 0.16923393),
  ('n01882714', 'koala', 0.14802812),
  ('n01883070', 'wombat', 0.113699146),
  ('n02108915', 'French_bulldog', 0.1092371),
  ('n02125311', 'cougar', 0.04562817)]]

In [13]:
for layer in densenet.layers:
    layer.trainable = False
    
for layer in densenet.layers[-10:]:
    layer.trainable = True


In [14]:
model_cats_HW = tf.keras.models.Sequential([
    densenet,
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [15]:
model_cats_HW.summary(line_length=200)

Model: "sequential"
________________________________________________________________________________________________________________________________________________________________________________________________________
 Layer (type)                                                                             Output Shape                                                                    Param #                       
 densenet201 (Functional)                                                                 (None, 1000)                                                                    20242984                      
                                                                                                                                                                                                        
 dense (Dense)                                                                            (None, 1)                                                                       1001  

In [16]:
accuracy = tf.keras.metrics.binary_accuracy
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
auc = tf.keras.metrics.AUC()

def f1_metrics(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+1e-7))

model_cats_HW.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[accuracy,
                       precision,
                       recall,
                       f1_metrics])              

In [17]:
def preprocces_image(file):
    img = tf.keras.preprocessing.image.load_img(file, target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.resnet.preprocess_input(img)
    return img

In [18]:
import os

cats = [(preprocces_image('pics/cats/'+file), 1) for file in os.listdir('pics/cats')]

nocats = [(preprocces_image('pics/nocats/'+file), 0) for file in os.listdir('pics/nocats')]

In [19]:
all_pics=cats+nocats

In [20]:
np.random.shuffle(all_pics)

In [21]:
np.shape(all_pics)

C:\Users\Vadim\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


(458, 2)

In [22]:
x = np.array([a[0] for a in all_pics])
y = np.array([a[1] for a in all_pics])

In [23]:
def train_val_test_split(x, val_frac=0.15, test_frac=0.15):
    x_train = x[:round((1-val_frac-test_frac)*len(x))]
    x_val = x[round((1-val_frac-test_frac)*len(x)):round((1-test_frac)*len(x))]
    x_test = x[round((1-test_frac)*len(x)):]
    return x_train, x_val, x_test

x_train, x_val, x_test = train_val_test_split(x)
y_train, y_val, y_test = train_val_test_split(y)

In [24]:
print(x_train.shape, x_val.shape, x_test.shape)

(321, 224, 224, 3) (68, 224, 224, 3) (69, 224, 224, 3)


In [25]:
print(y_train.shape, y_val.shape, y_test.shape)

(321,) (68,) (69,)


In [26]:
tb_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/tl_densenet_cats', histogram_freq=1)
    
annealing = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=10, verbose=1)

In [27]:
# tensorboard --logdir D:\ProductStar\Neural_networks\logs\tl_densenet_cats\

In [28]:
model_cats_HW.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          batch_size=16,
          epochs=50,
          callbacks=[tb_callback, annealing])

Epoch 1/50
21/21 [==============================] - 81s 3s/step - loss: 0.6871 - binary_accuracy: 0.5950 - precision: 0.4737 - recall: 0.1406 - f1_metrics: 0.2016 - val_loss: 0.6804 - val_binary_accuracy: 0.6471 - val_precision: 0.2222 - val_recall: 0.1053 - val_f1_metrics: 0.0806 - lr: 0.0010
Epoch 2/50
21/21 [==============================] - 67s 3s/step - loss: 0.6739 - binary_accuracy: 0.7664 - precision: 0.8118 - recall: 0.5391 - f1_metrics: 0.6314 - val_loss: 0.6741 - val_binary_accuracy: 0.6471 - val_precision: 0.3810 - val_recall: 0.4211 - val_f1_metrics: 0.2807 - lr: 0.0010
Epoch 3/50
21/21 [==============================] - 65s 3s/step - loss: 0.6660 - binary_accuracy: 0.7414 - precision: 0.7848 - recall: 0.4844 - f1_metrics: 0.5787 - val_loss: 0.6720 - val_binary_accuracy: 0.6471 - val_precision: 0.4194 - val_recall: 0.6842 - val_f1_metrics: 0.4019 - lr: 0.0010
Epoch 4/50
21/21 [==============================] - 65s 3s/step - loss: 0.6624 - binary_accuracy: 0.7726 - precisio

In [33]:
model_cats_HW.evaluate(x_val, y_val)

3/3 [==============================] - 9s 2s/step - loss: 0.5169 - binary_accuracy: 0.7794 - precision: 0.6000 - recall: 0.6316 - f1_metrics: 0.6107


[0.5168940424919128,
 0.779411792755127,
 0.6000000238418579,
 0.6315789222717285,
 0.6107304096221924]

In [31]:
model_cats_HW.evaluate(x_test, y_test)

3/3 [==============================] - 9s 2s/step - loss: 0.5420 - binary_accuracy: 0.7536 - precision: 0.7143 - recall: 0.5769 - f1_metrics: 0.6243


[0.5420404076576233,
 0.7536231875419617,
 0.7142857313156128,
 0.5769230723381042,
 0.6242645978927612]

In [32]:
model_cats_HW.predict(cat_img)

array([[0.2604371]], dtype=float32)